[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/llama-index/llama-index-intro.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/llama-index/llama-index-intro.ipynb)

# Llama-Index with Pinecone

In this notebook, we will demo how to use the `llama-index` (previously GPT-index) library with Pinecone for semantic search. This notebook is an introduction and does not cover the more advanced features of `llama-index`. You will find these in future releases in the [Pinecone examples repo](https://github.com/pinecone-io/examples/tree/master/learn/generation).

We will start by installing the necessary libraries and initializing Pinecone.

In [14]:
!pip install -qU llama-index datasets pinecone-client openai transformers

We can go ahead and load the **SQuAD** dataset, which contains questions and answer pairs from Wikipedia articles. We'll then convert the dataset into a pandas DataFrame and keep only the unique 'context' fields, which are the text passages that the questions are based on.

In [15]:
from datasets import load_dataset

data = load_dataset('squad', split='train')
data = data.to_pandas()[['id', 'context', 'title']]
data.drop_duplicates(subset='context', keep='first', inplace=True)
data.head()

,id,context,title
0,5733be284776f41900661182,"Architecturally, the school has a Catholic cha...",University_of_Notre_Dame
5,5733bf84d058e614000b61be,"As at most other universities, Notre Dame's st...",University_of_Notre_Dame
10,5733bed24776f41900661188,The university is the major seat of the Congre...,University_of_Notre_Dame
15,5733a6424776f41900660f51,The College of Engineering was established in ...,University_of_Notre_Dame
20,5733a70c4776f41900660f64,All of Notre Dame's undergraduate students are...,University_of_Notre_Dame


In [16]:
len(data)

18891

This code transforms our DataFrame into a list of Document objects, ready for indexing with llama_index. Each document contains the text passage, a unique id, and an extra field for the article title.

In [17]:
from llama_index import Document

docs = []

for i, row in data.iterrows():
    docs.append(Document(
        text=row['context'],
        doc_id=row['id'],
        extra_info={'title': row['title']}
    ))
docs[0]

Document(text='Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', doc_id='5733be284776f41900661182', embedding=None, doc_hash='ba8eb7d843763d4f7e7f71f0be64c70c4597fd712bfcce77007fc4c821b8d1b6', extra_info={'title': 'University_of_Notre_Dame'})

In [18]:
len(docs)

18891

Here, we're setting up the OpenAI API key and initializing a `SimpleNodeParser`. This parser processes our list of `Document` objects into 'nodes', which are the basic units that `llama_index` uses for indexing and querying. The first node is displayed below.

In [19]:
import os

os.environ['OPENAI_API_KEY'] = 'OPENAI_API_KEY'  # platform.openai.com

In [20]:
from llama_index.node_parser import SimpleNodeParser

parser = SimpleNodeParser()

nodes = parser.get_nodes_from_documents(docs)
nodes[0]

Node(text='Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', doc_id='8968aa3c-abbe-4fe0-ad91-be0298830542', embedding=None, doc_hash='ba8eb7d843763d4f7e7f71f0be64c70c4597fd712bfcce77007fc4c821b8d1b6', extra_info={'title': 'University_of_Notre_Dame'}, node_info={'start': 0, 'end': 695}, relationships={<DocumentRelationship.SOURCE: '1'>: '5733be284776

In [21]:
len(nodes)

18892

### Indexing in Pinecone

Pinecone is a managed vector database service designed for machine learning applications. We're using it in this context to store and retrieve embeddings generated by our language model, enabling efficient and scalable semantic similarity-based search.

We initialize Pinecone with the relevant API key and environment that we [get for **free** in the console](https://app.pinecone.io/), then create a new index. The index has a dimension of 1536 and uses cosine similarity, which is the recommended metric for comparing vectors produced by the `text-embedding-ada-002` model we'll be using.

In [22]:
import pinecone

# find API key in console at app.pinecone.io
os.environ['PINECONE_API_KEY'] = 'PINECONE_API_KEY'
# environment is found next to API key in the console
os.environ['PINECONE_ENVIRONMENT'] = 'PINECONE_ENVIRONMENT'

# initialize connection to pinecone
pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'],
    environment=os.environ['PINECONE_ENVIRONMENT']
)

# create the index if it does not exist already
index_name = 'llama-index-intro'
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=1536,
        metric='cosine'
    )

# connect to the index
pinecone_index = pinecone.Index(index_name)

Here, we're initializing a `PineconeVectorStore` with our previously created Pinecone index. This object will serve as the storage and retrieval interface for our document embeddings in Pinecone's vector database.

In [23]:
from llama_index.vector_stores import PineconeVectorStore

# we can select a namespace (acts as a partition in an index)
namespace = '' # default namespace

vector_store = PineconeVectorStore(pinecone_index=pinecone_index)

Next we initialize the `GPTVectorStoreIndex` with our list of `Document` objects, using the `PineconeVectorStore` as storage and `OpenAIEmbedding` model for embeddings.

`StorageContext` is used to configure the storage setup, and `ServiceContext` sets up the embedding model. The `GPTVectorStoreIndex` handles the indexing and querying process, making use of the provided storage and service contexts.

In [24]:
from llama_index import GPTVectorStoreIndex, StorageContext, ServiceContext
from llama_index.embeddings.openai import OpenAIEmbedding

# setup our storage (vector db)
storage_context = StorageContext.from_defaults(
    vector_store=vector_store
)
# setup the index/query process, ie the embedding model (and completion if used)
embed_model = OpenAIEmbedding(model='text-embedding-ada-002', embed_batch_size=100)
service_context = ServiceContext.from_defaults(embed_model=embed_model)

index = GPTVectorStoreIndex.from_documents(
    docs, storage_context=storage_context,
    service_context=service_context
)

Finally we can build a query engine from the `index` we build and use this engine to perform a query.

In [25]:
query_engine = index.as_query_engine()
res = query_engine.query("in what year was the college of engineering established at the University of Notre Dame?")
print(res)


The College of Engineering was established in 1920.


That's our quick intro to using Llama-index and Pinecone! Once we're done testing the system we should delete the Pinecone index to save resources:

In [26]:
pinecone.delete_index(index_name)

---